### Authenticate collab


In [ ]:
# from google.colab import files
# files.upload()  # Manually upload the JSON key file


In [7]:
from google.oauth2 import service_account
import google.auth

# Replace 'your-key-file.json' with your actual filename
creds = service_account.Credentials.from_service_account_file('rag-alloydb-poc-450115-2a2dd5ce28f8.json')
google.auth.default = lambda: (creds, None)


In [9]:
# If you're using these credentials for a specific API client, pass them explicitly
# Example: Using BigQuery
from google.cloud import bigquery
client = bigquery.Client(credentials=creds)
# Verify authentication
print(client.project)  # This should print the project ID

rag-alloydb-poc-450115


# AlloyDB Retreve Data

In [1]:
%pip install langchain-google-alloydb-pg langchain langchain-google-vertexai google-cloud-alloydb-connector[pg8000]


In [2]:

# Please fill in these values.
project_id = "rag-alloydb-poc-450115"

# Configure gcloud.
!gcloud config set project {project_id}

Updated property [core/project].


In [3]:

region = "us-central1"
cluster_name = "alloydb-cluster"
instance_name = "primary-instance"
database_name = "langchain"
password = input("Please provide a password to be used for 'postgres' database user: ")


Please provide a password to be used for 'postgres' database user: postgres


In [4]:
from google.colab import auth

auth.authenticate_user()

In [5]:
from langchain_google_alloydb_pg import AlloyDBChatMessageHistory, AlloyDBEngine
from langchain_google_alloydb_pg import AlloyDBVectorStore, AlloyDBEngine
from langchain_google_alloydb_pg import AlloyDBEngine, Column
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_alloydb_pg import AlloyDBVectorStore, AlloyDBEngine
from langchain_google_alloydb_pg import AlloyDBEngine, Column
from langchain_google_alloydb_pg import AlloyDBEngine, Column, AlloyDBLoader
import pandas as pd
from google.cloud.alloydb.connector import Connector, IPTypes
import sqlalchemy

In [6]:

engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)
message_table_name = "message_store"

engine.init_chat_history_table(table_name=message_table_name)

chat_history = AlloyDBChatMessageHistory.create_sync(
    engine,
    session_id="my-test-session",
    table_name=message_table_name,
)

In [7]:
chat_history.add_user_message("Hi!")
chat_history.add_ai_message("Hello there. I'm a model and am happy to help!")

chat_history.messages

[HumanMessage(content='in which year My Little Pony was released?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='My Little Pony was released in 1983. I found this information in the first sentence of the second paragraph of the text you provided.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the story of My Little Pony', additional_kwargs={}, response_metadata={}),
 AIMessage(content='My Little Pony has multiple stories, each with its own characters and settings. Here are some of the most popular ones:\n\n* **My Little Pony: Friendship Is Magic** takes place in the magical land of Equestria, where unicorn Twilight Sparkle and her friends go on adventures and learn valuable lessons about friendship.\n* **Equestria Girls** is set in an alternate universe where the ponies have become teenage girls who must learn what it means to be human while attending Canterlot High.\n* **My Little Pony: A New Generation** shows Equestria divided, but 

## Conversational RAG Chain backed by AlloyDB

In [8]:
from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_google_alloydb_pg import (
    AlloyDBEngine,
    AlloyDBVectorStore,
    AlloyDBChatMessageHistory,
)

sample_vector_table_name = "movie_vector_table_samples"

# Intialize the embedding service
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@003", project=project_id
)

# Intialize the engine
engine = AlloyDBEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    cluster=cluster_name,
    database=database_name,
    user="postgres",
    password=password,
)
# Intialize the Vector Store
vector_store = AlloyDBVectorStore.create_sync(
    engine=engine,
    embedding_service=embeddings_service,
    table_name=sample_vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

# Intialize the AlloyDBChatMessageHistory
chat_history = AlloyDBChatMessageHistory.create_sync(
    engine,
    session_id="my-test-session",
    table_name="message_store",
)

In [9]:
# Prepare some prompt templates for the ConversationalRetrievalChain
prompt = PromptTemplate(
    template="""Use all the information from the context and the conversation history to answer new question. If you see the answer in previous conversation history or the context. \
Answer it with clarifying the source information. If you don't see it in the context or the chat history, just say you \
didn't find the answer in the given data. Don't make things up.

Previous conversation history from the questioner. "Human" was the user who's asking the new question. "Assistant" was you as the assistant:
```{chat_history}
```

Vector search result of the new question:
```{context}
```

New Question:
```{question}```

Answer:""",
    input_variables=["context", "question", "chat_history"],
)
condense_question_prompt_passthrough = PromptTemplate(
    template="""Repeat the following question:
{question}
""",
    input_variables=["question"],
)

In [10]:
# Initialize retriever, llm and memory for the chain
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.8}
)

In [11]:
llm = VertexAI(model_name="gemini-pro", project=project_id)

In [12]:
chat_history.clear()

memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=chat_history,
    output_key="answer",
    memory_key="chat_history",
    return_messages=True,
)

<ipython-input-12-b6e34cd98860>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [13]:
# create the ConversationalRetrievalChain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    verbose=False,
    memory=memory,
    condense_question_prompt=condense_question_prompt_passthrough,
    combine_docs_chain_kwargs={"prompt": prompt},
)

# ask some questions
q = "What movie was Brad Pitt in?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "How about Jonny Depp?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "Are there movies about animals?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

# browser the chat history
chat_history.messages

<ipython-input-13-8710e06fe420>:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]


Question: What movie was Brad Pitt in?
Answer: Brad Pitt has acted in many movies. According to the search results, here are a few of the movies he has acted in:

* Inglourious Basterds
* By the Sea
* Killing Them Softly
* Babel

Is there a specific movie you are interested in? 


Question: How about Jonny Depp?
Answer: ## Johnny Depp's film roles:

Based on the provided information, here are some movies starring Johnny Depp:

* **The Rum Diary:** Johnny Depp plays a journalist who moves to Puerto Rico and falls in love with a taken woman.
* **Charlie and the Chocolate Factory:** Depp portrays the eccentric Willy Wonka in this adaptation of the classic children's book.
* **The Tourist:** He plays an American tourist who finds himself in danger after meeting a beautiful Interpol agent.
* **The Imaginarium of Doctor Parnassus:** Depp plays one of several actors who portray the titular character in this fantasy film.

These are just a few of the many films Johnny Depp has starred in. 

Wo

[HumanMessage(content='What movie was Brad Pitt in?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Brad Pitt has acted in many movies. According to the search results, here are a few of the movies he has acted in:\n\n* Inglourious Basterds\n* By the Sea\n* Killing Them Softly\n* Babel\n\nIs there a specific movie you are interested in? \n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How about Jonny Depp?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="## Johnny Depp's film roles:\n\nBased on the provided information, here are some movies starring Johnny Depp:\n\n* **The Rum Diary:** Johnny Depp plays a journalist who moves to Puerto Rico and falls in love with a taken woman.\n* **Charlie and the Chocolate Factory:** Depp portrays the eccentric Willy Wonka in this adaptation of the classic children's book.\n* **The Tourist:** He plays an American tourist who finds himself in danger after meeting a beautiful Interpol agent.\

In [14]:
chat_history.clear()

In [15]:
q = "in which year My Little Pony was released?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

Question: in which year My Little Pony was released?
Answer: My Little Pony was released in 1983. This information can be found in the previous responses from the user "Human".

I hope this clarifies things. Is there anything else I can help you with?



In [16]:
q = "what is the story of My Little Pony"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

Question: what is the story of My Little Pony
Answer: ## The Story of My Little Pony

My Little Pony is a long-running franchise with a rich history spanning several decades. It encompasses various TV shows, movies, toys, and other merchandise. The core theme throughout the franchise is friendship and the importance of understanding and appreciating different personalities.

Here's a brief overview of the different My Little Pony stories:

**My Little Pony (1983):** The first generation of My Little Pony toys and animated specials focused on a group of earth ponies living in the magical land of Ponyland. Their adventures revolved around exploring their world, helping others, and learning valuable lessons about friendship and kindness.

**My Little Pony: Friendship is Magic (2010):** This popular fourth generation of My Little Pony introduced a new generation of ponies with different types: unicorns, pegasi, and earth ponies. The story follows Twilight Sparkle, a studious unicorn who tr

In [17]:
q = "is Midnight Mass movie or web series"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

Question: is Midnight Mass movie or web series
Answer: ## "Midnight Mass" is a web series.

This information is not directly stated in the question or the provided context. However, I can find evidence supporting this claim in the vector search results:

* **Source:** The description of "Midnight Mass" in the vector search result explicitly states: **"a young priest brings glorious miracles, ominous mysteries and renewed religious fervor to a dying town desperate to believe."** This wording implies a narrative format consistent with a web series rather than a movie. Web series typically present serialized storytelling across multiple episodes, allowing for the gradual development of characters and intricate plotlines – aspects that seem fitting for the description provided.

While this evidence provides a strong indication that "Midnight Mass" is a web series, it's crucial to note that this response relies on the interpretation of the provided text and does not constitute absolute conf

In [18]:
q = "Tell me deatils about The Great British Baking Show"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

Question: Tell me deatils about The Great British Baking Show
Answer: ## Summary of The Great British Baking Show

Thanks for providing such a comprehensive overview of The Great British Baking Show. Here's a summary of the key points you mentioned:

**Format:**

* Amateur bakers compete in various baking challenges.
* Three challenges each episode: Signature Bake, Technical Challenge, and Showstopper Challenge.
* Judges assess technical skill, creativity, and presentation.
* Lowest-scoring baker eliminated each week.

**Challenges:**

* Signature Bake: Bakers showcase their own creations based on a theme.
* Technical Challenge: Identical recipes and ingredients, judged on execution.
* Showstopper Challenge: Bakers impress with creative and technical bakes.

**Judges:**

* Paul Hollywood: Renowned baker, known for rigorous judging.
* Prue Leith: Chef, restaurateur, extensive culinary knowledge.

**Production:**

* Love Productions, filmed in a large tent in the British countryside.
* H